In [1]:
%pip install spark-nlp==6.0.2 numpy pandas matplotlib seaborn setuptools wordcloud pypdf2 contractions unidecode
%pip install --upgrade setuptools

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import findspark
findspark.init('.Development/spark/spark-3.5.1-bin-hadoop3')
from pyspark import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DecimalType
from pyspark.sql.functions import regexp_extract, split, from_unixtime, col, avg, min, max
from pyspark.sql.functions import grouping_id, window, explode, to_json, from_json
from pyspark.sql.functions import udf, lit, current_timestamp, current_date, date_format
from pyspark.ml import PipelineModel, Pipeline
import PyPDF2
import os

In [3]:

spark = SparkSession.builder \
    .appName("nlpdemo") \
    .config("spark.jars.packages", 
            ",".join([
                "com.johnsnowlabs.nlp:spark-nlp-silicon_2.12:6.0.2"
            ])) \
    .config("spark.driver.memory", "8g")\
    .getOrCreate()

25/07/27 18:44:00 WARN Utils: Your hostname, Joses-MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 192.168.100.185 instead (on interface en0)
25/07/27 18:44:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/joseaguilar/.ivy2/cache
The jars for the packages stored in: /Users/joseaguilar/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp-silicon_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fdb7bf9c-ba07-4489-80b1-33cf1264d7b7;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp-silicon_2.12;6.0.2 in central


:: loading settings :: url = jar:file:/Users/joseaguilar/Documents/Development/spark/spark-3.5.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in local-m2-cache
	found org.apache.httpcomponents#httpclient;4.5.13 in local-m2-cache
	found org.apache.httpcomponents#httpcore;4.4.13 in local-m2-cache
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1

In [4]:
import sparknlp
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import Tokenizer, Normalizer

print(sparknlp.version())
spark = sparknlp.start(apple_silicon=True, memory="8G")

6.0.2


25/07/27 18:44:06 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [6]:
docs = [
    "El gobierno anunció nuevas medidas económicas en la capital.",
    "Los equipos de fútbol de Madrid y Barcelona jugaron un partido emocionante.",
    "Investigadores descubren una nueva especie de dinosaurio en Sudamérica.",
    "El mercado de acciones cerró a la baja tras noticias internacionales.",
    "Un nuevo avance tecnológico impulsa el desarrollo de inteligencia artificial.",
    "La película ganadora del Oscar es un drama histórico aclamado por la crítica.",
    "Se inauguró un nuevo museo de arte moderno en París.",
    "Científicos advierten sobre el cambio climático en conferencia global.",
    "El campeonato mundial de tenis tendrá lugar el próximo verano.",
    "Empresas tecnológicas reportan incrementos récord en sus ganancias trimestrales."
]
df_docs = spark.createDataFrame([(d,) for d in docs], ["texto"])
# Preprocesar: tokenizar, eliminar stopwords, etc.
from sparknlp.annotator import Tokenizer, StopWordsCleaner

tokenizer = Tokenizer().setInputCols(["documento"]).setOutputCol("token")
stopwords_cleaner = StopWordsCleaner()\
    .setInputCols(["token"]).setOutputCol("token_sin_stop")\
    .setCaseSensitive(False)\
    .setStopWords(["y", "de", "la", "the", "and", "is", "es", "porque"])  # lista combinada EN/ES como ejemplo

pipeline_stop = Pipeline(stages=[DocumentAssembler().setInputCol("texto").setOutputCol("documento"),
                                 tokenizer, stopwords_cleaner,
                                 Finisher().setInputCols(["token_sin_stop"]).setOutputCols(["tokens_final"])])

df_tokens = pipeline_stop.fit(df_docs).transform(df_docs)
df_tokens.select("texto", "tokens_final").show(3, truncate=100)


+---------------------------------------------------------------------------+-------------------------------------------------------------------------------+
|                                                                      texto|                                                                   tokens_final|
+---------------------------------------------------------------------------+-------------------------------------------------------------------------------+
|               El gobierno anunció nuevas medidas económicas en la capital.|           [El, gobierno, anunció, nuevas, medidas, económicas, en, capital, .]|
|Los equipos de fútbol de Madrid y Barcelona jugaron un partido emocionante.|[Los, equipos, fútbol, Madrid, Barcelona, jugaron, un, partido, emocionante, .]|
|    Investigadores descubren una nueva especie de dinosaurio en Sudamérica.|[Investigadores, descubren, una, nueva, especie, dinosaurio, en, Sudamérica, .]|
+---------------------------------------------------

In [7]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="tokens_final", outputCol="features", vocabSize=1000, minDF=1)
cv_model = cv.fit(df_tokens)
df_feats = cv_model.transform(df_tokens)
df_feats.select("features").show(1, truncate=False)


+-------------------------------------------------------------------+
|features                                                           |
+-------------------------------------------------------------------+
|(79,[0,1,4,8,27,31,34,51,65],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
+-------------------------------------------------------------------+
only showing top 1 row



In [8]:
from pyspark.ml.clustering import LDA
num_topics = 3
lda = LDA(k=num_topics, seed=42, featuresCol="features")
lda_model = lda.fit(df_feats)


25/07/27 18:46:00 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/07/27 18:46:00 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [9]:
topics = lda_model.describeTopics(maxTermsPerTopic=5)
topics.show(truncate=False)


+-----+--------------------+-----------------------------------------------------------------------------------------------------------+
|topic|termIndices         |termWeights                                                                                                |
+-----+--------------------+-----------------------------------------------------------------------------------------------------------+
|0    |[43, 22, 58, 12, 68]|[0.01599150857333902, 0.015044653089289196, 0.0146410180725598, 0.014512515810278247, 0.014492569150778139]|
|1    |[0, 4, 1, 48, 74]   |[0.02695003323093981, 0.0205887251035435, 0.018447919210700135, 0.01804487973918214, 0.017959994036253295] |
|2    |[0, 2, 71, 25, 18]  |[0.02874070803410647, 0.02630213696391263, 0.020526205925635435, 0.02004911674923326, 0.020015301096214078]|
+-----+--------------------+-----------------------------------------------------------------------------------------------------------+



In [11]:
# Collect vocabulary as a plain Python list to avoid serialization issues
vocab = list(cv_model.vocabulary)
topics_rdd = topics.rdd
topics_words = topics_rdd.map(lambda row: [vocab[idx] for idx in row['termIndices']]).collect()
for idx, words in enumerate(topics_words):
    print(f"Tema {idx}: {words}")


Tema 0: ['mundial', 'drama', 'inteligencia', 'Se', 'récord']
Tema 1: ['.', 'El', 'en', 'tras', 'internacionales']
Tema 2: ['.', 'el', 'impulsa', 'avance', 'artificial']


In [12]:
df_topics = lda_model.transform(df_feats)
df_topics.select("texto", "topicDistribution").show(truncate=False)


+--------------------------------------------------------------------------------+--------------------------------------------------------------+
|texto                                                                           |topicDistribution                                             |
+--------------------------------------------------------------------------------+--------------------------------------------------------------+
|El gobierno anunció nuevas medidas económicas en la capital.                    |[0.029699840488906927,0.9323830724066302,0.037917087104462906]|
|Los equipos de fútbol de Madrid y Barcelona jugaron un partido emocionante.     |[0.028868005251978267,0.9335746031173442,0.03755739163067755] |
|Investigadores descubren una nueva especie de dinosaurio en Sudamérica.         |[0.03135626653927194,0.03784720381712324,0.9307965296436048]  |
|El mercado de acciones cerró a la baja tras noticias internacionales.           |[0.026614536674784247,0.9399380692882486,0

In [14]:
import numpy as np
asignar_tema = udf(lambda vec: int(np.argmax(vec)), "int")
df_topics = df_topics.withColumn("tema_predominante", asignar_tema(col("topicDistribution")))
df_topics.select("texto", "tema_predominante").show(truncate=False)


+--------------------------------------------------------------------------------+-----------------+
|texto                                                                           |tema_predominante|
+--------------------------------------------------------------------------------+-----------------+
|El gobierno anunció nuevas medidas económicas en la capital.                    |1                |
|Los equipos de fútbol de Madrid y Barcelona jugaron un partido emocionante.     |1                |
|Investigadores descubren una nueva especie de dinosaurio en Sudamérica.         |2                |
|El mercado de acciones cerró a la baja tras noticias internacionales.           |1                |
|Un nuevo avance tecnológico impulsa el desarrollo de inteligencia artificial.   |2                |
|La película ganadora del Oscar es un drama histórico aclamado por la crítica.   |0                |
|Se inauguró un nuevo museo de arte moderno en París.                            |1        